## *To perform the Test:*

**Connection Setup:**
   - Connect two Supernovas between them via their HV bus and each one to the computer via the USB port. One will act as an I3C controller and the other one as an I3C target.

   The target Supernova acts as a 2 bytes-addressable and non-linear memory of 1024 bytes. The memory can be accessed via USB when the user requests an i3cTargetWriteMemory or i3cTargetReadMemory directly to the target. Or it can be accessed through the I3C bus when the user requests an I3C write, I3C read or I3C write-read to the controller targeting the target.

**Expected Messages and Notifications:**
   - For each command directly sent to the controller, the notebook should show a the response returned by it.
   - For each command directly sent to the target, the notebook should show a the response returned by it.
   - For each request of an I3C write / read / write-read sent (via USB) to the controller but targeting the Supernova target, the notebook should show the response from the controller and a notification from the target
   
**Testing Process:**
   - Run each cell in order.
   - Look for immediate feedback:
     - The response or the notification and response for each command directed to a Supernova.


### 1. Startup

#### 1.1 Import the python package

In [1]:
from BinhoSupernova import getConnectedSupernovaDevicesList
from BinhoSupernova.Supernova import Supernova
from BinhoSupernova.commands.definitions import *
import threading

print_lock = threading.Lock()

#### 1.2 List all the Supernova devices connected to the PC host

The ``BinhoSupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

Two different Supernovas with different serial numbers and paths must appear in the response.

In [2]:
getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#6&2fcfc8d8&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': 'D77B1F7C300FAC5184484347207B87D',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': -1},
 {'path': '\\\\?\\HID#VID_1FC9&PID_82FC#7&f321146&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '59314AC593A1C05C8163B6759994476',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': -1}]

#### 1.3 Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[0, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [3]:
# Auxiliar code to generate IDs

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 1.4 Define and register callbacks to handle responses and notifications from the two Supernovas (the one acting as an I3C target and the one acting as an I3C controller)

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [4]:
import json

# Define callback function
def controller_callback_function(supernova_message: dict, system_message: dict) -> None:

    with print_lock:
    
        if supernova_message != None:

                # Print a header
                print(">> New message from controller SUPERNOVA:")

                # If the command is GET TARGET DEVICE TABLE, convert to json.
                if supernova_message["name"] == "I3C GET TARGET DEVICE TABLE" or supernova_message["name"] == "I3C TRANSFER":
                    responseJson = json.dumps(supernova_message, indent=4)
                    print(responseJson)

                else:
                    print(supernova_message)
        
        if system_message != None:

            # Print a header
            print(">> New message from the SYSTEM:")

            print(system_message)

In [5]:
# Define callback function
def target_callback_function(supernova_message: dict, system_message: dict) -> None:

    with print_lock:

        if supernova_message != None:

                # Print a header
                print(">> New message from target SUPERNOVA:")

                print(supernova_message)
        
        if system_message != None:

            # Print a header
            print(">> New message from the SYSTEM:")

            print(system_message)

### 2. Configure the Supernova that will act as an I3C target

#### 2.1 Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [7]:
supernovaTarget = Supernova()

#### 2.2 Open connection to the Supernova that acts as a target using the path of one of the devices retrieved by getConnectedSupernovaDevicesList() 

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [8]:
supernovaTarget.open(path ='\\\\?\\HID#VID_1FC9&PID_82FC#6&2fcfc8d8&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 2.3 Register the callback to the I3C target

In [9]:
supernovaTarget.onEvent(target_callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 2.4 Initialize the Supernova I3C peripheral as a target memory with a register size of 1 byte

In [10]:
SECONDS_TO_WAIT_FOR_IBI = 0x69
MAX_READ_LENGTH = 0x100
MAX_WRITE_LENGTH = 0x100

features =  DdrOk.ALLOWED_DDR.value |   \
            IgnoreTE0TE1Errors.IGNORE_ERRORS.value |   \
            MatchStartStop.NOT_MATCH.value |   \
            AlwaysNack.NOT_ALWAYS_NACK.value 

request_result = supernovaTarget.i3cTargetInit(getId(), I3cTargetMemoryLayout_t.MEM_1_BYTE, SECONDS_TO_WAIT_FOR_IBI, MAX_READ_LENGTH, MAX_WRITE_LENGTH, features)

>> New message from target SUPERNOVA:
{'id': 1, 'command': 16, 'name': 'I3C TARGET INIT', 'result': 'I3C_TARGET_INIT_SUCCESS'}


#### 2.5 Set the target Supernova PID

In [11]:
PID = [0x02, 0x03, 0x04, 0x05, 0x06, 0x07]

request_result = supernovaTarget.i3cTargetSetPid(getId(), PID)

>> New message from target SUPERNOVA:
{'id': 2, 'command': 21, 'name': 'I3C TARGET SET PID', 'result': 'I3C_TARGET_SET_PID_SUCCESS'}


#### 2.6 Set the target Supernova BCR

In [12]:
request_result = supernovaTarget.i3cTargetSetBcr(getId(), I3cTargetMaxDataSpeedLimit_t.MAX_DATA_SPEED_LIMIT, I3cTargetIbiCapable_t.NOT_IBI_CAPABLE, I3cTargetIbiPayload_t.IBI_WITH_PAYLOAD, I3cTargetOfflineCap_t.OFFLINE_CAPABLE, I3cTargetVirtSupport_t.VIRTUAL_TARGET_SUPPORT, I3cTargetDeviceRole_t.I3C_TARGET)

>> New message from target SUPERNOVA:
{'id': 3, 'command': 22, 'name': 'I3C TARGET SET BCR', 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_MGR_NO_ERROR', 'driver_result': 'I3C_TARGET_DRIVER_NO_ERROR'}


#### 2.7 Set the target Supernova DCR

In [13]:
request_result = supernovaTarget.i3cTargetSetDcr(getId(), I3cTargetDcr_t.I3C_TARGET_MEMORY)

>> New message from target SUPERNOVA:
{'id': 4, 'command': 23, 'name': 'I3C TARGET SET DCR', 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_MGR_NO_ERROR', 'driver_result': 'I3C_TARGET_DRIVER_NO_ERROR'}


#### 2.8 Set the target Supernova Static Address

In [14]:
request_result = supernovaTarget.i3cTargetSetStaticAddr(getId(), 0x73)

>> New message from target SUPERNOVA:
{'id': 5, 'command': 24, 'name': 'I3C TARGET SET STATIC ADDRESS', 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_MGR_NO_ERROR', 'driver_result': 'I3C_TARGET_DRIVER_NO_ERROR'}


### 3. Configure the Supernova that will act as an I3C controller

#### 3.1 Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [15]:
supernovaController = Supernova()

#### 3.2 Open connection to the Supernova that acts as a controller using the path of one of the devices retrieved by getConnectedSupernovaDevicesList() 

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [16]:
supernovaController.open(path ='\\\\?\\HID#VID_1FC9&PID_82FC#7&f321146&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 3.3 Register the callback to the I3C controller

In [17]:
supernovaController.onEvent(controller_callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 3.4 Initialize the Supernova I3C peripheral as controller

In [18]:
request_result = supernovaController.i3cControllerInit(getId())

>> New message from controller SUPERNOVA:
{'id': 6, 'command': 1, 'name': 'I3C CONTROLLER INIT', 'result': 'I3C_CONTROLLER_INIT_SUCCESS'}


#### 3.5 I3C bus voltage

By default, both Supernvas work at 1.62V. If it is neccessary to change the bus voltage, the voltage must be set in both devices. Currently we are adding the feature in the firmware so that the Supernova as an I3C Target will be able to set the voltage automatically base on the voltage applied in the VTARG pin.

In [ ]:
#Uncomment lines below if it is required to change the I3C bus voltage.

#supernovaController.setI3cBusVoltage(getId(), 3300)
#supernovaTarget.setI3cBusVoltage(getId(), 3300)

#### 3.6 I3C bus initialization

In [19]:
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cRSTDAA(getId(), PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from controller SUPERNOVA:
{
    "id": 7,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0
    ]
}


In [20]:
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cSETDASA(getId(), 0x73, 0x08, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 8,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0
    ]
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFI

In [21]:
request_result = supernovaController.i3cGetTargetDeviceTable(getId())

>> New message from controller SUPERNOVA:
{
    "id": 9,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 0,
    "table": []
}


#### 3.7 I3C CCCs

##### 3.7.1 GETPID

In [22]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cGETPID(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from controller SUPERNOVA:
{
    "id": 10,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 6,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "pid": [
        "0x07",
        "0x06",
        "0x05",
        "0x04",
        "0x03",
        "0x02"
    ]
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}


##### 3.7.2 GETBCR

In [23]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ
    
request_result = supernovaController.i3cGETBCR(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 11,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "bcr": {
        "value": [
            "0b00111101",
            61,
            "0x3d"
        ],
        "description": {
            "deviceRole": "I3C Target.",
            "advancedCapabilities": "Supports optional advanced capabilities. Use GETCAPS CCC to deremine which ones.",
            "virtualTargetSupport": "Is a Virtual Target, or exposes other downstream Device(

##### 3.7.3 GETDCR

In [24]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cGETDCR(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 12,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "dcr": "0xc5"
}


##### 3.7.4 GETMWL

In [25]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cGETMWL(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 13,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "maxWriteLength": 256
}


##### 3.7.5 SETMWL

In [26]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ
NEW_MWL         = 1024

request_result = supernovaController.i3cDirectSETMWL(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE, NEW_MWL)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 14,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0
    ]
}


##### 3.7.6 GETMWL

In [27]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cGETMWL(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from controller SUPERNOVA:
{
    "id": 15,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "maxWriteLength": 1024
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}


##### 3.7.7 GETMRL

In [28]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cGETMRL(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 16,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "maxReadLength": 256,
    "maxIbiPayloadSize": [
        0,
        0,
        0,
        0,
        0
    ]
}


##### 3.7.8 SETMRL

In [29]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ
NEW_MRL         = 512

request_result = supernovaController.i3cDirectSETMRL(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE, NEW_MRL)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 17,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0
    ]
}


##### 3.7.9 GETMRL

In [30]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cGETMRL(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 18,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "maxReadLength": 512,
    "maxIbiPayloadSize": [
        0,
        0,
        0,
        0,
        0
    ]
}


##### 3.7.10 DISEC

In [31]:
TARGET_ADDRESS  = 0x08

request_result = supernovaController.i3cClearFeature(getId(), I3cClearFeatureSelector.REGULAR_IBI, TARGET_ADDRESS)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{'id': 19, 'command': 7, 'name': 'I3C CLEAR FEATURE', 'result': 'I3C_CLEAR_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


##### 3.7.11 ENEC

In [32]:
TARGET_ADDRESS  = 0x08

request_result = supernovaController.i3cSetFeature(getId(), I3cSetFeatureSelector.REGULAR_IBI, TARGET_ADDRESS)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{'id': 20, 'command': 8, 'name': 'I3C SET FEATURE', 'result': 'I3C_SET_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


##### 3.7.12 GET STATUS

In [33]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cGETSTATUS(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 21,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0
    ]
}


#### 3.8 I3C TARGET SET CONFIGURATION

The public method ``Supernova.i3cTargetSetConfiguration()`` sets the configuration of the supernova acting as a target on the I3C bus.

```python
i3cTargetSetConfiguration(id, uSecondsToWaitForIbi, maxReadLength, maxWriteLength, targetFeatures)
```

- ``id: c_uint16``: Identifies the command.
- ``uSecondsToWaitForIbi: c_uint8``: Micro seconds to allow an In-Band Interrupt (IBI) to drive SDA low when the controller is not doing so.
- ``maxReadLength: c_uint16``: Maximum read length that the user wants the Supernova to handle.
- ``maxWriteLength: c_uint16``: Maximum write length that the user wants the Supernova to handle.
- ``targetFeatures: c_uint8``: Series of flags that describe the features of the Supernova in target mode.

In [34]:
MICRO_SECONDS_TO_WAIT_FOR_IBI = 0x69
MAX_READ_LENGTH = 0x300 # 768
MAX_WRITE_LENGTH = 0x250 # 592

features =  DdrOk.ALLOWED_DDR.value |  \
            IgnoreTE0TE1Errors.IGNORE_ERRORS.value |  \
            MatchStartStop.NOT_MATCH.value |  \
            AlwaysNack.NOT_ALWAYS_NACK.value

request_result = supernovaTarget.i3cTargetSetConfiguration(getId(), MICRO_SECONDS_TO_WAIT_FOR_IBI, MAX_READ_LENGTH, MAX_WRITE_LENGTH, features)

>> New message from target SUPERNOVA:
{'id': 22, 'command': 17, 'name': 'I3C TARGET SET CONFIGURATION', 'result': 'I3C_TARGET_SET_CONF_SUCCESS'}


##### 3.8.1 Check the correct setting of the MWL during i3cTargetSetConfiguration requesting a GETMWL

In [35]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cGETMWL(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{
    "id": 23,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "maxWriteLength": 592
}


##### 3.8.2 Check the correct setting of the MRL during i3cTargetSetConfiguration requesting a GETMRL

In [36]:
TARGET_ADDRESS  = 0x08
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ

request_result = supernovaController.i3cGETMRL(getId(), TARGET_ADDRESS, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from controller SUPERNOVA:
{
    "id": 24,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "maxReadLength": 768,
    "maxIbiPayloadSize": [
        0,
        0,
        0,
        0,
        0
    ]
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}


#### 3.9 I3C TARGET MEMORY RELATED COMMANDS WITH THE TARGET ACTING AS A MEMORY WITH A REGISTER SIZE OF 1 BYTE

##### 3.9.1 I3C TARGET WRITE MEMORY

The public method ``Supernova.i3cTargetWriteMemory()`` writes the memory of the target Supernova using only USB communication.

```python
i3cTargetWriteMemory(id, memoryAddr, data)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddr: c_uint16``: Address of the memory to start writing.
- ``data: list``: Data to write.

In [37]:
MEMORY_ADDRESS  = 0x0000
DATA            = [i%0xFA for i in range(0,1024)]

request_result = supernovaTarget.i3cTargetWriteMemory(getId(), MEMORY_ADDRESS, DATA)

>> New message from target SUPERNOVA:
{'id': 25, 'command': 18, 'name': 'I3C TARGET WRITE MEMORY', 'result': 'I3C_TARGET_WRITE_MEM_SUCCESS', 'error': 'NO_ERROR'}


##### 3.9.2 I3C TARGET READ MEMORY

The public method ``Supernova.i3cTargetReadMemory()`` reads the memory of the target Supernova using only USB communication.

```python
i3cTargetReadMemory(id, memoryAddr, length)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddr: c_uint16``: Address of the memory to start reading.
- ``length: c_uint16``: Data length the user intends to read.

In [38]:
MEMORY_ADDRESS  = 0x0000
DATA_LENGTH     = 1024 # in bytes

request_result = supernovaTarget.i3cTargetReadMemory(getId(), MEMORY_ADDRESS, DATA_LENGTH)

>> New message from target SUPERNOVA:
{'id': 26, 'command': 19, 'name': 'I3C TARGET READ MEMORY', 'result': 'I3C_TARGET_READ_MEM_SUCCESS', 'error': 'NO_ERROR', 'readDataLength': 1024, 'data': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 18

#### 3.10 I3C WRITE AND READ

##### 3.10.1 I3C WRITE INDICATING THE START ADDRESS AND THE DATA

In [39]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00, 0x00]
DATA            = [0xEE for i in range(5)]

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 0, 'transfer_length': 5, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': [238, 238, 238, 238, 238]}
>> New message from controller SUPERNOVA:
{
    "id": 27,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


##### 3.10.2 I3C READ WITHOUT INDICATING THE START ADDRESS

The target will assume that the start of the new operation is the end of the previous one

In [40]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 10

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_WRITE', 'target_address': 8, 'memory_address': 5, 'transfer_length': 10, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['ABORT_CONDITION'], 'data': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]}
>> New message from controller SUPERNOVA:
{
    "id": 28,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 10,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14
    ]
}


##### 3.10.3 I3C WRITE-READ

A write followed by a read with a RS in between

In [41]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xBC, 0x02] # 700
LENGTH          = 5 # in bytes

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_WRITE', 'target_address': 8, 'memory_address': 700, 'transfer_length': 5, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['ABORT_CONDITION'], 'data': [200, 201, 202, 203, 204]}
>> New message from controller SUPERNOVA:
{
    "id": 29,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 5,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        200,
        201,
        202,
        203,
        204
    ]
}


##### 3.10.4 I3C WRITE INDICATING THE START ADDRESS FOLLOWED BY A READ WITHOUT ADDRESS

In [42]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xF4, 0x03] # 1012
DATA            = []

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 1012, 'transfer_length': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': []}
>> New message from controller SUPERNOVA:
{
    "id": 30,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


In [43]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 6

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from controller SUPERNOVA:
{
    "id": 31,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 6,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        12,
        13,
        14,
        15,
        16,
        17
    ]
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_WRITE', 'target_address': 8, 'memory_address': 1012, 'transfer_length': 6, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['ABORT_CONDITION'], 'data': [12, 13, 14, 15, 16, 17]}


#### 3.10.5 BORDER CASES

##### 3.10.5.1 USER TRIES TO START A TRANSFER SURPASSING THE MEMORY RANGE

The target will ignore the bytes of address and will start the transfer from the end of the the previous one.

In [44]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x01, 0x04] # 1025
DATA            = [0xEE for i in range(4)]

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from controller SUPERNOVA:
{
    "id": 32,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 1018, 'transfer_length': 4, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': [238, 238, 238, 238]}


##### 3.10.5.2 THE TRANSFER STARTS IN AN ALLOWED ADDRESS BUT TRIES TO SURPASS THE MEMORY RANGE ON THE GO

It will only modify the bytes in the allowed range and discard the rest. The end of the transfer is taken as the end of the memory

In [45]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xFC, 0x03] # 1020
DATA            = []

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from controller SUPERNOVA:
{
    "id": 33,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 1020, 'transfer_length': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': []}


In [46]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 30

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_WRITE', 'target_address': 8, 'memory_address': 1020, 'transfer_length': 4, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': [238, 238, 22, 23]}
>> New message from controller SUPERNOVA:
{
    "id": 34,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 30,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        238,
        238,
        22,
        23,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    

#### 3.11 Initialize the Supernova I3C peripheral as a target memory with a register size of 4 bytes

In [47]:
MICRO_SECONDS_TO_WAIT_FOR_IBI = 0x69
MAX_READ_LENGTH = 0x100
MAX_WRITE_LENGTH = 0x100

features =  DdrOk.ALLOWED_DDR.value |   \
            IgnoreTE0TE1Errors.IGNORE_ERRORS.value |   \
            MatchStartStop.NOT_MATCH.value |   \
            AlwaysNack.NOT_ALWAYS_NACK.value 

request_result = supernovaTarget.i3cTargetInit(getId(), I3cTargetMemoryLayout_t.MEM_4_BYTES, MICRO_SECONDS_TO_WAIT_FOR_IBI, MAX_READ_LENGTH, MAX_WRITE_LENGTH, features)

>> New message from target SUPERNOVA:
{'id': 35, 'command': 16, 'name': 'I3C TARGET INIT', 'result': 'I3C_TARGET_INIT_SUCCESS'}


#### 3.12 Init the bus

In [48]:
request_result = supernovaController.i3cInitBus(getId())

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from controller SUPERNOVA:
{'id': 36, 'command': 2, 'name': 'I3C INIT BUS', 'result': 'I3C_BUS_INIT_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_CCC', 'target_address': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_ADDR_CHANGED', 'new_address': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR']}


#### 3.13 I3C TARGET MEMORY RELATED COMMANDS WITH THE TARGET ACTING AS A MEMORY WITH A REGISTER SIZE OF 4 BYTES

In [49]:
MEMORY_ADDRESS  = 0x0000
DATA            = [i%0xFA for i in range(0,1024)]

request_result = supernovaTarget.i3cTargetWriteMemory(getId(), MEMORY_ADDRESS, DATA)

>> New message from target SUPERNOVA:
{'id': 37, 'command': 18, 'name': 'I3C TARGET WRITE MEMORY', 'result': 'I3C_TARGET_WRITE_MEM_SUCCESS', 'error': 'NO_ERROR'}


##### 3.13.1 I3C TARGET WRITE MEMORY

The public method ``Supernova.i3cTargetWriteMemory()`` writes the memory of the target Supernova using only USB communication.

```python
i3cTargetWriteMemory(id, memoryAddr, data)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddr: c_uint16``: Address of the memory to start writing.
- ``data: list``: Data to write.

In [50]:
MEMORY_ADDRESS  = 0x0A
DATA            = [0xFF for i in range(0,8)]

request_result = supernovaTarget.i3cTargetWriteMemory(getId(), MEMORY_ADDRESS, DATA)

>> New message from target SUPERNOVA:
{'id': 38, 'command': 18, 'name': 'I3C TARGET WRITE MEMORY', 'result': 'I3C_TARGET_WRITE_MEM_SUCCESS', 'error': 'NO_ERROR'}


##### 3.13.2 I3C TARGET READ MEMORY

The public method ``Supernova.i3cTargetReadMemory()`` reads the memory of the target Supernova using only USB communication.

```python
i3cTargetReadMemory(id, memoryAddr, length)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddr: c_uint16``: Address of the memory to start reading.
- ``length: c_uint16``: Data length the user intends to read.

In [51]:
MEMORY_ADDRESS  = 0x00
DATA_LENGTH     = 1024 # in bytes

request_result = supernovaTarget.i3cTargetReadMemory(getId(), MEMORY_ADDRESS, DATA_LENGTH)

>> New message from target SUPERNOVA:
{'id': 39, 'command': 19, 'name': 'I3C TARGET READ MEMORY', 'result': 'I3C_TARGET_READ_MEM_SUCCESS', 'error': 'NO_ERROR', 'readDataLength': 1024, 'data': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 255, 255, 255, 255, 255, 255, 255, 255, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,

#### 3.14 I3C WRITE AND READ

##### 3.14.1 I3C WRITE INDICATING THE START ADDRESS AND THE DATA

In [52]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x3B] # register 59 = byte 236
DATA            = [0xEE for i in range(8)]

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 59, 'transfer_length': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': [238, 238, 238, 238, 238, 238, 238, 238]}
>> New message from controller SUPERNOVA:
{
    "id": 40,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


##### 3.14.2 I3C READ WITHOUT INDICATING THE START ADDRESS

The target will assume that the start of the new operation is the end of the previous one

In [53]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 4

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from controller SUPERNOVA:
{
    "id": 41,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 4,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        244,
        245,
        246,
        247
    ]
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_WRITE', 'target_address': 8, 'memory_address': 61, 'transfer_length': 4, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['ABORT_CONDITION'], 'data': [244, 245, 246, 247]}


##### 3.14.3 I3C WRITE-READ

A write followed by a read with a RS in between

In [54]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x7D] # register 125 = byte 500
LENGTH          = 8 # in bytes

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from controller SUPERNOVA:
{
    "id": 42,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 8,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        1,
        2,
        3,
        4,
        5,
        6,
        7
    ]
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_WRITE', 'target_address': 8, 'memory_address': 125, 'transfer_length': 8, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['ABORT_CONDITION'], 'data': [0, 1, 2, 3, 4, 5, 6, 7]}


##### 3.14.4 I3C WRITE INDICATING THE START ADDRESS FOLLOWED BY A READ WITHOUT ADDRESS

In [55]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xFE] # register 254 = byte 1016  
DATA            = []

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 254, 'transfer_length': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': []}
>> New message from controller SUPERNOVA:
{
    "id": 43,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


In [56]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 4

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_WRITE', 'target_address': 8, 'memory_address': 254, 'transfer_length': 4, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['ABORT_CONDITION'], 'data': [16, 17, 18, 19]}
>> New message from controller SUPERNOVA:
{
    "id": 44,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 4,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        16,
        17,
        18,
        19
    ]
}


#### 3.14.5 BORDER CASES

##### 3.14.5.1 THE TRANSFER STARTS IN AN ALLOWED ADDRESS BUT TRIES TO SURPASS THE MEMORY RANGE ON THE GO

It will only modify the bytes in the allowed range and discard the rest. The end of the transfer is taken as the end of the memory

In [57]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xFD] # register 253 = byte 1012
DATA            = []

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from controller SUPERNOVA:
{
    "id": 45,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 253, 'transfer_length': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': []}


In [58]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 40

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from controller SUPERNOVA:
{
    "id": 46,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 40,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20,
        21,
        22,
        23,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0
    ]
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_WRITE', 'target_address': 8, 'memory_addr

#### 3.15 LONG TRANSFERS

##### 3.15.1 I3C WRITE INDICATING THE START ADDRESS AND THE DATA

In [59]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
DATA            = [0x47 for i in range(254)] + [0x07] + [0x37 for i in range(769)]

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from controller SUPERNOVA:
{
    "id": 47,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 0, 'transfer_length': 255, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['MRL_REACHED'], 'data': [71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 

##### 3.15.2 I3C WRITE-READ

A write followed by a read with a RS in between

In [60]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
LENGTH          = 255 # in bytes

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from controller SUPERNOVA:
{
    "id": 48,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 255,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
      

##### 3.15.3 I3C WRITE INDICATING THE START ADDRESS FOLLOWED BY A READ WITHOUT ADDRESS

In [61]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
DATA            = []

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from controller SUPERNOVA:
{
    "id": 49,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 0, 'transfer_length': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': []}


In [62]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
LENGTH          = 255 # in bytes

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from controller SUPERNOVA:
{
    "id": 50,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 255,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
        71,
      

##### 3.15.3 I3C WRITE INDICATING THE START ADDRESS FOLLOWED BY A READ WITHOUT ADDRESS

In [63]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
DATA            = []

request_result = supernovaController.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from controller SUPERNOVA:
{
    "id": 51,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}
>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_READ', 'target_address': 8, 'memory_address': 0, 'transfer_length': 0, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['NO_ERROR'], 'data': []}


In [64]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 255

request_result = supernovaController.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

>> New message from target SUPERNOVA:
{'id': 0, 'command': 20, 'name': 'I3C TARGET NOTIFICATION', 'notification_type': 'I3C_TARGET_WRITE', 'target_address': 8, 'memory_address': 0, 'transfer_length': 255, 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_TARGET_TRANSFER_SUCCESS', 'driver_result': ['ABORT_CONDITION'], 'data': [71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71,